In [2]:
!pip install -q -U google-generativeai

import google.generativeai as genai
from IPython.display import Markdown, display
from google.colab import userdata
import textwrap



In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda l: True))

try:
  GOOGLE_API_KEY = "XxXxXxXxXxXxXxXxXxXxXxXxXxXx"
  genai.configure(api_key=GOOGLE_API_KEY)
  print("Gemini API key loaded successfully from Colab Secrets!")
except Exception as e:
  print(f"Error loading API key: {e}")
  print("Please make sure you have added GOOGLE_API_KEY to Colab Secrets and enabled 'Notebook access'.")
  raise ValueError("Google API Key not found or accessible. Please check Colab Secrets.")



Gemini API key loaded successfully from Colab Secrets!


In [4]:
try:
  text_generation_model_name = None
  print("\n--- Listing available models ---")
  available_models = list(genai.list_models())

  for m in available_models:
    print(f"  - Model: {m.name}, Supported Methods: {m.supported_generation_methods}")
    if 'generateContent' in m.supported_generation_methods:
      if 'gemini-1.5-flash-latest' in m.name:
        text_generation_model_name = m.name
        break
      elif 'gemini-1.5-pro-latest' in m.name:
        text_generation_model_name = m.name
      elif 'gemini-1.5-pro' in m.name and text_generation_model_name is None:
        text_generation_model_name = m.name
      elif 'gemini-1.5-flash' in m.name and text_generation_model_name is None:
        text_generation_model_name = m.name
      elif 'gemini-pro' in m.name and text_generation_model_name is None:
          text_generation_model_name = m.name
      elif 'gemma' in m.name and text_generation_model_name is None:
          text_generation_model_name = m.name

  if text_generation_model_name is None:
    raise ValueError("No suitable text generation model found. Please check your access and available models.")

  print(f"\nUsing model: {text_generation_model_name} for summarization.")
  model = genai.GenerativeModel(text_generation_model_name)

except Exception as e:
  print(f"Error listing or selecting models: {e}")
  print("This might be due to API key issues, network problems, or no suitable models being available.")
  raise




--- Listing available models ---
  - Model: models/embedding-gecko-001, Supported Methods: ['embedText', 'countTextTokens']
  - Model: models/gemini-1.0-pro-vision-latest, Supported Methods: ['generateContent', 'countTokens']
  - Model: models/gemini-pro-vision, Supported Methods: ['generateContent', 'countTokens']
  - Model: models/gemini-1.5-pro-latest, Supported Methods: ['generateContent', 'countTokens']
  - Model: models/gemini-1.5-pro-001, Supported Methods: ['generateContent', 'countTokens', 'createCachedContent']
  - Model: models/gemini-1.5-pro-002, Supported Methods: ['generateContent', 'countTokens', 'createCachedContent']
  - Model: models/gemini-1.5-pro, Supported Methods: ['generateContent', 'countTokens']
  - Model: models/gemini-1.5-flash-latest, Supported Methods: ['generateContent', 'countTokens']

Using model: models/gemini-1.5-flash-latest for summarization.


In [8]:
def summarize_text(text, max_length=150):
  if not text or len(text.strip()) == 0:
    return "Error: No text provided for summarization."

  prompt = f"Provide a brief and impactful summary of the following movie plot. Focus on the core premise, key events, major turning points, and the ultimate outcome. Aim for around {max_length} words.\n\nPlot Synopsis:\n{text}\n\nSummary:"

  try:
    response = model.generate_content(prompt)
    if response.parts:
        return response.text
    else:
        print("Warning: Model generated an empty or non-textual response. Check safety settings or content.")
        if response.prompt_feedback:
            if response.prompt_feedback.block_reason:
                print(f"Prompt was blocked due to: {response.prompt_feedback.block_reason.name}")
            if response.prompt_feedback.safety_ratings:
                print("Safety ratings for prompt:")
                for rating in response.prompt_feedback.safety_ratings:
                    print(f"- {rating.category.name}: {rating.probability.name}")
        if response.candidates:
            for candidate in response.candidates:
                if candidate.finish_reason:
                    print(f"Candidate finish reason: {candidate.finish_reason.name}")
                if candidate.safety_ratings:
                    print("Safety ratings for candidate:")
                    for rating in candidate.safety_ratings:
                        print(f"- {rating.category.name}: {rating.probability.name}")
        return "Error: No summary generated. The model might have been blocked or returned an empty response."
  except Exception as e:
    print(f"An error occurred during summarization: {e}")
    return f"An error occurred during summarization: {e}"



In [9]:
long_text = """
In 2008, in Kenya, the Special Anti-Terrorism Squad (SATS), consisting of M. S. Gandhi and his teammates Sunil Thiagarajan, Kalyan Sundaram and Ajay Govindharaj, intercept the terrorist Omar to extract uranium from him. Their former chief, Rajiv Menon, absconding after being charged with treason, is also on the same train. They retrieve the uranium from Menon and destroy the train he was on, believing everyone on board, including Menon, to have died. Back in Delhi, the team lives undercover as employees of a tourism agency, hiding their real jobs from their families. Gandhi lives with his wife, Anuradha "Anu", who is pregnant, and their five-year-old son, Jeevan.

Gandhi takes his family along to alleviate Anu's suspicions of infidelity when he is assigned a new mission in Bangkok. After completing the mission, Gandhi and his family are attacked, leading to Anu experiencing labour and Jeevan mysteriously disappearing. The police find a charred body believed to be Jeevan's, leaving Gandhi devastated. Anu delivers a baby girl but stops speaking to Gandhi out of grief.

By 2024, Gandhi has left SATS and is working as an immigration officer at Chennai International Airport. He and Anu live separately, and their daughter, Jeevitha, lives with Anu. Upon Kalyan's suggestion, Gandhi's former boss, Nazeer, asks him to go to Moscow to train new officers at the reopened Indian Embassy. Though reluctant, Gandhi agrees, but during his time there, the embassy is attacked by a gang. Gandhi fights them off and is shocked to see one of the attackers, a young doppelgänger. He realizes it is Jeevan and confirms this when he helps him fend off the attackers. They return to Chennai, and the family reunites, with Anu starting to speak to Gandhi again. Nazeer calls Gandhi for an urgent meeting at the Chennai Metro station, stating that the information is so confidential it cannot be told over the phone. A masked man attacks Nazeer. Gandhi arrives just in time to fight off the attacker, but the masked man kills Nazeer. The masked figure is later revealed to be Jeevan and Gandhi swears to find the killer and take revenge for Naseer’s death.

Gandhi returns to SATS to find who is trying to cause a threat to their agency. Nazeer's phone has all the information about Jeevan and his gang from Moscow. Jeevan's accomplice Sha took the phone from Nazeer in the Metro station when Nazeer dropped it. But Sha reveals to Jeevan that the phone went missing. It is found that Jeevan has been working for Menon all this time and has come to Gandhi's family for a reason.

It is also revealed that Menon survived the train explosion in 2008 but lost his wife and children in the incident. Seeking revenge, he tracked Gandhi to Bangkok, kidnapped Jeevan, and placed him in a juvenile facility, instilling fear in him. He then "rescued" Jeevan by killing the members of the facility, earning his trust and renaming him Sanjay after his deceased son. Jeevan, manipulated by Menon, came to believe Gandhi was responsible for the deaths of his family. Fueled by this hatred, Jeevan dedicated his life to Menon and his revenge on Gandhi.

Diamond Babu, a thief, steals Nazeer's phone, containing key evidence. Gandhi tries to buy it, but Jeevan intervenes. Ajay tries to stop Jeevan but is murdered. Abdul and Menon's arrest prompts Jeevan to kidnap Srinidhi, blackmailing Sunil to secure their release. Kalyan requests Gandhi to free them. Sunil witnessing that Jeevan had slashed Srinidhi's throat requests Kalyan to release them and he does so. After they reach the spot Kalyan suffocates her to death prevent her from becoming a key witness, revealing that he had been working with Menon all along. Sunil devastated about his daughter’s death swears revenge once he finds out who was the killer. While tracking down Sha, Gandhi coincidentally finds Jeevan and arrests him. During interrogation, he seizes Sunil's gun, seemingly kills him and forces Gandhi to escort him out, allowing Jeevan to escape. However, Sunil is revealed to be alive and kills Kalyan taking revenge for Srinidhi’s death. Menon kidnaps Anu and reveals a bombing plan, unaware it is Gandhi's ploy. Gandhi captures Menon and brings him to the M. A. Chidambaram Stadium to negotiate with Jeevan to prevent the bombing. Jeevan plans to detonate multiple bombs at the stadium and frame Gandhi for treason. Gandhi, with the help of his brother-in-law Seenu and Jeevitha, prevents the detonation. Jeevan threatens Jeevitha's life, but Gandhi kills him.

It is later revealed that "Jeevan" was a clone created by the real Jeevan, still known as Sanjay, who has since created more clones to target Gandhi.
"""

print("\n--- Original Text ---")
display(to_markdown(long_text))

print("\n--- Generated Summary ---")
summary = summarize_text(long_text, max_length=80)
display(to_markdown(summary))

print("\n--- Another Example with a different text ---")
another_text = """
Two men waiting at the Biju Patnaik Airport (in Bhubaneswar, Odisha, for a flight to Chennai, Tamil Nadu) engage in conversation. One is a commercial director, Anbarasu, who prefers the abbreviated name A. Aras, and the other is a scarred and deformed socialist, Nallasivam, going by Mr. Sivam. When the flight is cancelled due to heavy rain, Aras initially suspects Sivam to be a terrorist and informs the authorities, only to discover that he was mistaken. With the rain flooding the city, the two men are forced to share a room for the night. Both need to return to Chennai: Aras to be present at his wedding, and Sivam has to deliver a ₹32,00,000[a] cheque, recently awarded to him after he won a court case, to a group of union workers.

After a traumatic night, and no hope for a flight, the two men take a bus to board the Coromandel Express train. Aras's bag gets stolen on the way, leaving him with only his credit card, which no one accepts. Using his presence of mind, Sivam repeatedly bails Aras out of trouble while Aras tries to escape from him at every juncture, only to end up with him again. While waiting for the train at the Ichchapuram railway station (in Andhra Pradesh), Sivam begins to tell Aras his story.

A few years earlier, a healthy Nallasivam, going by Nalla, took part in various street theatre performances protesting against multinational corporation-driven industrialisation, which resulted in the marginalisation of the labour force. He was at odds with Kandasamy Padayatchi, a manipulative factory owner who refused to give his workers a raise. Nalla satirically imitated Padayatchi in many of his shows. In an unexpected turn of events, Nalla and Balasaraswathi (Bala), Padayatchi's daughter, fell in love with each other. To avoid a potential conflict with Padayatchi, the two decided to elope to Kerala.

Nalla boarded a bus bound for Kerala, and on his way to meet Bala, the bus met with an accident on a hillside, leaving him scarred, disfigured, and partially paralysed for life. After recovering from his wounds, he visited Bala, only to be informed by Padayatchi that his daughter was already married and pregnant with her first child, and had settled abroad. Padayatchi had earlier lied to Bala that Nalla died in the accident. It was also at this time that Nalla became a firm believer in kindness and love after experiencing patience and compassion from the nurses at the hospital. Despite suffering from an inferiority complex due to his scarred and deformed body, Nalla performs community service and social work with renewed fervour while continuing to fight for union causes.

Upon their arrival in Chennai, Aras delivers Sivam's cheque to the union workers. He invites Sivam to his wedding, letting him know how much he means to him, to utter astonishment, Sivam sees that Aras's bride is Bala. Padayatchi spots Sivam and asks him why he is at the wedding. He tells Padayatchi that he was invited by Aras, and later persuades him to sign the papers which will help Padayatchi's workers get a raise. To prevent the disruption of Bala's wedding and avoid damaging his own reputation, Padayatchi yields to Sivam's demands. After signing the papers, Padayatchi instructs his assistant to eliminate Sivam. However, his assistant has a change of heart as he is about to kill him, letting Sivam know that he believed that the misdeeds he committed for Padayatchi resulted in the death of his own daughter. Padayatchi's assistant requests Sivam to stay as far away from his boss as possible. Sivam assents and walks away.
"""

print("\n--- Original Text ---")
display(to_markdown(another_text))

print("\n--- Generated Summary (shorter) ---")
summary_2 = summarize_text(another_text, max_length=50)
display(to_markdown(summary_2))


--- Original Text ---


> 
> In 2008, in Kenya, the Special Anti-Terrorism Squad (SATS), consisting of M. S. Gandhi and his teammates Sunil Thiagarajan, Kalyan Sundaram and Ajay Govindharaj, intercept the terrorist Omar to extract uranium from him. Their former chief, Rajiv Menon, absconding after being charged with treason, is also on the same train. They retrieve the uranium from Menon and destroy the train he was on, believing everyone on board, including Menon, to have died. Back in Delhi, the team lives undercover as employees of a tourism agency, hiding their real jobs from their families. Gandhi lives with his wife, Anuradha "Anu", who is pregnant, and their five-year-old son, Jeevan.
> 
> Gandhi takes his family along to alleviate Anu's suspicions of infidelity when he is assigned a new mission in Bangkok. After completing the mission, Gandhi and his family are attacked, leading to Anu experiencing labour and Jeevan mysteriously disappearing. The police find a charred body believed to be Jeevan's, leaving Gandhi devastated. Anu delivers a baby girl but stops speaking to Gandhi out of grief.
> 
> By 2024, Gandhi has left SATS and is working as an immigration officer at Chennai International Airport. He and Anu live separately, and their daughter, Jeevitha, lives with Anu. Upon Kalyan's suggestion, Gandhi's former boss, Nazeer, asks him to go to Moscow to train new officers at the reopened Indian Embassy. Though reluctant, Gandhi agrees, but during his time there, the embassy is attacked by a gang. Gandhi fights them off and is shocked to see one of the attackers, a young doppelgänger. He realizes it is Jeevan and confirms this when he helps him fend off the attackers. They return to Chennai, and the family reunites, with Anu starting to speak to Gandhi again. Nazeer calls Gandhi for an urgent meeting at the Chennai Metro station, stating that the information is so confidential it cannot be told over the phone. A masked man attacks Nazeer. Gandhi arrives just in time to fight off the attacker, but the masked man kills Nazeer. The masked figure is later revealed to be Jeevan and Gandhi swears to find the killer and take revenge for Naseer’s death.
> 
> Gandhi returns to SATS to find who is trying to cause a threat to their agency. Nazeer's phone has all the information about Jeevan and his gang from Moscow. Jeevan's accomplice Sha took the phone from Nazeer in the Metro station when Nazeer dropped it. But Sha reveals to Jeevan that the phone went missing. It is found that Jeevan has been working for Menon all this time and has come to Gandhi's family for a reason.
> 
> It is also revealed that Menon survived the train explosion in 2008 but lost his wife and children in the incident. Seeking revenge, he tracked Gandhi to Bangkok, kidnapped Jeevan, and placed him in a juvenile facility, instilling fear in him. He then "rescued" Jeevan by killing the members of the facility, earning his trust and renaming him Sanjay after his deceased son. Jeevan, manipulated by Menon, came to believe Gandhi was responsible for the deaths of his family. Fueled by this hatred, Jeevan dedicated his life to Menon and his revenge on Gandhi.
> 
> Diamond Babu, a thief, steals Nazeer's phone, containing key evidence. Gandhi tries to buy it, but Jeevan intervenes. Ajay tries to stop Jeevan but is murdered. Abdul and Menon's arrest prompts Jeevan to kidnap Srinidhi, blackmailing Sunil to secure their release. Kalyan requests Gandhi to free them. Sunil witnessing that Jeevan had slashed Srinidhi's throat requests Kalyan to release them and he does so. After they reach the spot Kalyan suffocates her to death prevent her from becoming a key witness, revealing that he had been working with Menon all along. Sunil devastated about his daughter’s death swears revenge once he finds out who was the killer. While tracking down Sha, Gandhi coincidentally finds Jeevan and arrests him. During interrogation, he seizes Sunil's gun, seemingly kills him and forces Gandhi to escort him out, allowing Jeevan to escape. However, Sunil is revealed to be alive and kills Kalyan taking revenge for Srinidhi’s death. Menon kidnaps Anu and reveals a bombing plan, unaware it is Gandhi's ploy. Gandhi captures Menon and brings him to the M. A. Chidambaram Stadium to negotiate with Jeevan to prevent the bombing. Jeevan plans to detonate multiple bombs at the stadium and frame Gandhi for treason. Gandhi, with the help of his brother-in-law Seenu and Jeevitha, prevents the detonation. Jeevan threatens Jeevitha's life, but Gandhi kills him.
> 
> It is later revealed that "Jeevan" was a clone created by the real Jeevan, still known as Sanjay, who has since created more clones to target Gandhi.



--- Generated Summary ---


> In 2008, anti-terrorist agent Gandhi intercepts a terrorist, unknowingly saving his future enemy, Rajiv Menon.  Years later, Gandhi's son, Jeevan, disappears, presumed dead.  In 2024, Gandhi reunites with a seemingly resurrected Jeevan, only to discover he's been manipulated by the surviving Menon into a revenge plot.  Betrayals and deaths ensue as Gandhi uncovers a conspiracy involving clones, revealing Jeevan as a pawn in Menon's scheme.  A final confrontation at a stadium ends with Gandhi killing Jeevan, but the threat of more clones remains.



--- Another Example with a different text ---

--- Original Text ---


> 
> Two men waiting at the Biju Patnaik Airport (in Bhubaneswar, Odisha, for a flight to Chennai, Tamil Nadu) engage in conversation. One is a commercial director, Anbarasu, who prefers the abbreviated name A. Aras, and the other is a scarred and deformed socialist, Nallasivam, going by Mr. Sivam. When the flight is cancelled due to heavy rain, Aras initially suspects Sivam to be a terrorist and informs the authorities, only to discover that he was mistaken. With the rain flooding the city, the two men are forced to share a room for the night. Both need to return to Chennai: Aras to be present at his wedding, and Sivam has to deliver a ₹32,00,000[a] cheque, recently awarded to him after he won a court case, to a group of union workers.
> 
> After a traumatic night, and no hope for a flight, the two men take a bus to board the Coromandel Express train. Aras's bag gets stolen on the way, leaving him with only his credit card, which no one accepts. Using his presence of mind, Sivam repeatedly bails Aras out of trouble while Aras tries to escape from him at every juncture, only to end up with him again. While waiting for the train at the Ichchapuram railway station (in Andhra Pradesh), Sivam begins to tell Aras his story.
> 
> A few years earlier, a healthy Nallasivam, going by Nalla, took part in various street theatre performances protesting against multinational corporation-driven industrialisation, which resulted in the marginalisation of the labour force. He was at odds with Kandasamy Padayatchi, a manipulative factory owner who refused to give his workers a raise. Nalla satirically imitated Padayatchi in many of his shows. In an unexpected turn of events, Nalla and Balasaraswathi (Bala), Padayatchi's daughter, fell in love with each other. To avoid a potential conflict with Padayatchi, the two decided to elope to Kerala.
> 
> Nalla boarded a bus bound for Kerala, and on his way to meet Bala, the bus met with an accident on a hillside, leaving him scarred, disfigured, and partially paralysed for life. After recovering from his wounds, he visited Bala, only to be informed by Padayatchi that his daughter was already married and pregnant with her first child, and had settled abroad. Padayatchi had earlier lied to Bala that Nalla died in the accident. It was also at this time that Nalla became a firm believer in kindness and love after experiencing patience and compassion from the nurses at the hospital. Despite suffering from an inferiority complex due to his scarred and deformed body, Nalla performs community service and social work with renewed fervour while continuing to fight for union causes.
> 
> Upon their arrival in Chennai, Aras delivers Sivam's cheque to the union workers. He invites Sivam to his wedding, letting him know how much he means to him, to utter astonishment, Sivam sees that Aras's bride is Bala. Padayatchi spots Sivam and asks him why he is at the wedding. He tells Padayatchi that he was invited by Aras, and later persuades him to sign the papers which will help Padayatchi's workers get a raise. To prevent the disruption of Bala's wedding and avoid damaging his own reputation, Padayatchi yields to Sivam's demands. After signing the papers, Padayatchi instructs his assistant to eliminate Sivam. However, his assistant has a change of heart as he is about to kill him, letting Sivam know that he believed that the misdeeds he committed for Padayatchi resulted in the death of his own daughter. Padayatchi's assistant requests Sivam to stay as far away from his boss as possible. Sivam assents and walks away.



--- Generated Summary (shorter) ---


> Stranded together after a flight cancellation, a cynical director and a scarred socialist embark on a chaotic journey.  Their initial distrust evolves into unlikely camaraderie as the socialist repeatedly rescues the director from escalating crises.  The journey culminates at the director's wedding, where the socialist's past love, now the bride, is revealed, leading to a confrontation with her manipulative father and a surprising resolution.
